# 📚 **Ingestão Completa de PDFs Longos - Pipeline RAG Completo**

> *Do PDF bruto até o banco vetorial pronto para busca - tudo em um notebook!*

---

## 🎯 **O que você vai aprender?**

Este notebook demonstra um **pipeline completo** para ingestão de PDFs longos, incluindo:

1. ✅ **Extração de texto** de PDFs (múltiplos arquivos)
2. ✅ **Criação de chunks** (semânticos e fixos)
3. ✅ **Geração de embeddings** 
4. ✅ **Armazenamento** em múltiplos bancos vetoriais:
   - 🗄️ **Chroma** (local, rápido)
   - 🚀 **FAISS** (local, escalável)
   - 📊 **LanceDB** (moderno, eficiente)

---

## 📋 **Pré-requisitos**

- Python básico
- Pasta `pdfs/` com seus PDFs para processar
- Paciência (processamento pode demorar para PDFs grandes)

---

**🖼️ Sugestão de imagem**: Um pipeline mostrando PDF → Chunks → Embeddings → Vector Store

## 🚀 **Setup Inicial - Instalando Dependências**

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB/LOCAL
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install pypdf>=3.15.0
!pip install huggingface_hub>=0.19.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install lancedb>=0.4.0
!pip install pyarrow>=12.0.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0
!pip install tiktoken>=0.5.0

print("✅ Dependências instaladas com sucesso!")
print("🚀 Pronto para processar PDFs longos!")

## 📦 **Importações Necessárias**

In [ ]:
# 📦 IMPORTAÇÕES PARA INGESTÃO DE PDFs
import os
from pathlib import Path
from typing import List, Dict, Any, Optional
import time

# LangChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# Vector Stores
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
import lancedb
import pyarrow as pa

# Utilitários
import numpy as np
import pandas as pd

print("✅ Bibliotecas importadas com sucesso!")
print("📚 Pronto para processar PDFs!")

## 📄 **Etapa 1: Extração de Texto de PDFs**

### **1.1: Carregando PDFs Individualmente**

In [ ]:
# 📄 ETAPA 1: EXTRAÇÃO DE TEXTO DE PDFs

def carregar_pdf(pdf_path: str) -> List[Document]:
    """
    Carrega um PDF e extrai o texto
    
    Args:
        pdf_path: Caminho para o arquivo PDF
    
    Returns:
        Lista de documentos do LangChain
    """
    try:
        print(f"📖 Carregando PDF: {Path(pdf_path).name}")
        
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        
        # Adicionar metadados úteis
        for i, doc in enumerate(documents):
            doc.metadata['source_file'] = Path(pdf_path).name
            doc.metadata['page_number'] = i + 1
            doc.metadata['total_pages'] = len(documents)
        
        print(f"✅ Carregado: {len(documents)} páginas")
        print(f"📝 Total de caracteres: {sum(len(d.page_content) for d in documents):,}")
        
        return documents
        
    except Exception as e:
        print(f"❌ Erro ao carregar {pdf_path}: {e}")
        return []

# Exemplo de uso
pdf_folder = Path("pdfs")

if pdf_folder.exists():
    pdf_files = list(pdf_folder.glob("*.pdf"))
    
    if pdf_files:
        print(f"📚 Encontrados {len(pdf_files)} arquivos PDF")
        print("\n" + "="*50)
        
        # Carregar primeiro PDF como exemplo
        primeiro_pdf = pdf_files[0]
        documentos_exemplo = carregar_pdf(str(primeiro_pdf))
        
        if documentos_exemplo:
            print("\n📄 Primeira página de exemplo:")
            print("-" * 50)
            print(documentos_exemplo[0].page_content[:500] + "...")
    else:
        print(f"⚠️ Nenhum PDF encontrado em {pdf_folder}")
        print("💡 Crie uma pasta 'pdfs' e coloque seus PDFs lá!")
else:
    print(f"⚠️ Pasta {pdf_folder} não existe")
    print("💡 Crie uma pasta 'pdfs' e coloque seus PDFs lá!")

### **1.2: Carregando Múltiplos PDFs**

In [ ]:
# 📚 CARREGANDO MÚLTIPLOS PDFs

def carregar_todos_pdfs(pdf_folder: str = "pdfs") -> List[Document]:
    """
    Carrega todos os PDFs de uma pasta
    
    Args:
        pdf_folder: Caminho para a pasta com PDFs
    
    Returns:
        Lista de todos os documentos
    """
    pdf_folder_path = Path(pdf_folder)
    
    if not pdf_folder_path.exists():
        print(f"❌ Pasta {pdf_folder} não encontrada!")
        return []
    
    pdf_files = list(pdf_folder_path.glob("*.pdf"))
    
    if not pdf_files:
        print(f"⚠️ Nenhum PDF encontrado em {pdf_folder}")
        return []
    
    print(f"📚 Encontrados {len(pdf_files)} arquivos PDF")
    print("="*60)
    
    all_documents = []
    
    for pdf_file in pdf_files:
        print(f"\n📖 Processando: {pdf_file.name}")
        
        documentos = carregar_pdf(str(pdf_file))
        
        if documentos:
            all_documents.extend(documentos)
            print(f"✅ Adicionado: {len(documentos)} páginas")
    
    print("\n" + "="*60)
    print(f"✅ Total: {len(all_documents)} páginas de {len(pdf_files)} PDFs")
    print(f"📝 Total de caracteres: {sum(len(d.page_content) for d in all_documents):,}")
    
    return all_documents

# Carregar todos os PDFs
todos_documentos = carregar_todos_pdfs("pdfs")

if todos_documentos:
    print("\n🎉 PDFs carregados com sucesso!")
    print(f"📊 Estatísticas:")
    print(f"   - Total de páginas: {len(todos_documentos)}")
    print(f"   - Média de caracteres por página: {sum(len(d.page_content) for d in todos_documentos) // len(todos_documentos)}")
    print(f"   - Arquivos únicos: {len(set(d.metadata.get('source_file', '') for d in todos_documentos))}")

## ✂️ **Etapa 2: Criação de Chunks (Semânticos e Fixos)**

### **2.1: Chunks Fixos (Character-based)**

In [ ]:
# ✂️ ETAPA 2.1: CHUNKS FIXOS (CHARACTER-BASED)

def criar_chunks_fixos(documents: List[Document], 
                       chunk_size: int = 1000, 
                       chunk_overlap: int = 200) -> List[Document]:
    """
    Cria chunks de tamanho fixo (caracteres)
    
    Args:
        documents: Lista de documentos
        chunk_size: Tamanho máximo de cada chunk (caracteres)
        chunk_overlap: Sobreposição entre chunks
    
    Returns:
        Lista de chunks
    """
    print(f"🔪 Criando chunks fixos (tamanho: {chunk_size}, overlap: {chunk_overlap})")
    
    splitter = CharacterTextSplitter(
        separator="\n\n",  # Separador preferencial
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    chunks = splitter.split_documents(documents)
    
    print(f"✅ Criados {len(chunks)} chunks")
    print(f"📏 Tamanho médio: {sum(len(c.page_content) for c in chunks) // len(chunks)} caracteres")
    print(f"📊 Tamanho mínimo: {min(len(c.page_content) for c in chunks)} caracteres")
    print(f"📊 Tamanho máximo: {max(len(c.page_content) for c in chunks)} caracteres")
    
    return chunks

# Criar chunks fixos (se tiver documentos)
if todos_documentos:
    chunks_fixos = criar_chunks_fixos(
        todos_documentos,
        chunk_size=1000,
        chunk_overlap=200
    )
    
    print("\n📄 Exemplo de chunk fixo:")
    print("-" * 50)
    print(chunks_fixos[0].page_content[:300] + "...")
else:
    print("⚠️ Carregue PDFs primeiro!")

### **2.2: Chunks Semânticos (Recursive Character-based)**

In [ ]:
# 🧠 ETAPA 2.2: CHUNKS SEMÂNTICOS (RECURSIVE CHARACTER-BASED)

def criar_chunks_semanticos(documents: List[Document], 
                            chunk_size: int = 1000, 
                            chunk_overlap: int = 200) -> List[Document]:
    """
    Cria chunks semânticos respeitando a estrutura do texto
    
    Args:
        documents: Lista de documentos
        chunk_size: Tamanho máximo de cada chunk
        chunk_overlap: Sobreposição entre chunks
    
    Returns:
        Lista de chunks semânticos
    """
    print(f"🧠 Criando chunks semânticos (tamanho: {chunk_size}, overlap: {chunk_overlap})")
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        # Ordem de prioridade para separadores
        separators=["\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " ", ""]
    )
    
    chunks = splitter.split_documents(documents)
    
    print(f"✅ Criados {len(chunks)} chunks semânticos")
    print(f"📏 Tamanho médio: {sum(len(c.page_content) for c in chunks) // len(chunks)} caracteres")
    print(f"📊 Tamanho mínimo: {min(len(c.page_content) for c in chunks)} caracteres")
    print(f"📊 Tamanho máximo: {max(len(c.page_content) for c in chunks)} caracteres")
    
    return chunks

# Criar chunks semânticos (se tiver documentos)
if todos_documentos:
    chunks_semanticos = criar_chunks_semanticos(
        todos_documentos,
        chunk_size=1000,
        chunk_overlap=200
    )
    
    print("\n📄 Exemplo de chunk semântico:")
    print("-" * 50)
    print(chunks_semanticos[0].page_content[:300] + "...")
    
    print("\n📊 Comparação:")
    print(f"   - Chunks fixos: {len(chunks_fixos) if 'chunks_fixos' in locals() else 0}")
    print(f"   - Chunks semânticos: {len(chunks_semanticos)}")
else:
    print("⚠️ Carregue PDFs primeiro!")

### **2.3: Chunks Baseados em Tokens**

In [ ]:
# 🎯 ETAPA 2.3: CHUNKS BASEADOS EM TOKENS

def criar_chunks_tokens(documents: List[Document], 
                        chunk_size: int = 512, 
                        chunk_overlap: int = 50) -> List[Document]:
    """
    Cria chunks baseados em tokens (útil para modelos de linguagem)
    
    Args:
        documents: Lista de documentos
        chunk_size: Número máximo de tokens por chunk
        chunk_overlap: Sobreposição de tokens
    
    Returns:
        Lista de chunks baseados em tokens
    """
    print(f"🎯 Criando chunks baseados em tokens (tamanho: {chunk_size}, overlap: {chunk_overlap})")
    
    try:
        splitter = TokenTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            encoding_name="cl100k_base"  # Encoding do GPT
        )
        
        chunks = splitter.split_documents(documents)
        
        print(f"✅ Criados {len(chunks)} chunks baseados em tokens")
        print(f"📏 Tamanho médio: {sum(len(c.page_content) for c in chunks) // len(chunks)} caracteres")
        
        return chunks
        
    except Exception as e:
        print(f"⚠️ Erro ao criar chunks baseados em tokens: {e}")
        print("💡 Usando chunks semânticos como alternativa")
        return criar_chunks_semanticos(documents, chunk_size=chunk_size*2, chunk_overlap=chunk_overlap*2)

# Criar chunks baseados em tokens (se tiver documentos)
if todos_documentos:
    chunks_tokens = criar_chunks_tokens(
        todos_documentos,
        chunk_size=512,
        chunk_overlap=50
    )
    
    print("\n📄 Exemplo de chunk baseado em tokens:")
    print("-" * 50)
    print(chunks_tokens[0].page_content[:300] + "...")
else:
    print("⚠️ Carregue PDFs primeiro!")

## 🧠 **Etapa 3: Geração de Embeddings**

In [ ]:
# 🧠 ETAPA 3: GERAÇÃO DE EMBEDDINGS

def configurar_embeddings(modelo: str = "sentence-transformers/all-MiniLM-L6-v2"):
    """
    Configura o modelo de embeddings
    
    Args:
        modelo: Nome do modelo de embeddings
    
    Returns:
        Objeto de embeddings configurado
    """
    print(f"🔧 Configurando embeddings: {modelo}")
    
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name=modelo,
            model_kwargs={'device': 'cpu'},  # Use 'cuda' se tiver GPU
            encode_kwargs={'normalize_embeddings': True}
        )
        
        print(f"✅ Embeddings configurados!")
        print(f"📊 Dimensão dos embeddings: 384 (para all-MiniLM-L6-v2)")
        
        return embeddings
        
    except Exception as e:
        print(f"❌ Erro ao configurar embeddings: {e}")
        return None

# Configurar embeddings
embeddings = configurar_embeddings()

# Testar embeddings (se tiver chunks)
if 'chunks_semanticos' in locals() and chunks_semanticos and embeddings:
    print("\n🧪 Testando geração de embeddings:")
    
    # Gerar embedding de um chunk de exemplo
    texto_teste = chunks_semanticos[0].page_content[:500]
    
    inicio = time.time()
    embedding_teste = embeddings.embed_query(texto_teste)
    tempo = time.time() - inicio
    
    print(f"✅ Embedding gerado em {tempo:.3f}s")
    print(f"📊 Dimensão: {len(embedding_teste)}")
    print(f"📊 Valores de exemplo: {embedding_teste[:5]}")

## 🗄️ **Etapa 4: Armazenamento em Bancos Vetoriais**

### **4.1: Armazenamento no Chroma**

In [ ]:
# 🗄️ ETAPA 4.1: ARMAZENAMENTO NO CHROMA

def armazenar_chroma(chunks: List[Document], 
                    embeddings: HuggingFaceEmbeddings,
                    collection_name: str = "pdfs_chroma",
                    persist_directory: str = "./chroma_db") -> Chroma:
    """
    Armazena chunks no Chroma
    
    Args:
        chunks: Lista de chunks
        embeddings: Modelo de embeddings
        collection_name: Nome da coleção
        persist_directory: Diretório para persistência
    
    Returns:
        Objeto Chroma configurado
    """
    print(f"🗄️ Armazenando {len(chunks)} chunks no Chroma...")
    
    inicio = time.time()
    
    try:
        vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            collection_name=collection_name,
            persist_directory=persist_directory
        )
        
        tempo = time.time() - inicio
        
        print(f"✅ Armazenado no Chroma em {tempo:.2f}s")
        print(f"📊 Coleção: {collection_name}")
        print(f"💾 Persistido em: {persist_directory}")
        
        return vectorstore
        
    except Exception as e:
        print(f"❌ Erro ao armazenar no Chroma: {e}")
        return None

# Armazenar no Chroma (se tiver chunks e embeddings)
if 'chunks_semanticos' in locals() and chunks_semanticos and embeddings:
    print("\n" + "="*60)
    vectorstore_chroma = armazenar_chroma(
        chunks_semanticos,
        embeddings,
        collection_name="pdfs_longos_chroma"
    )
    
    # Testar busca
    if vectorstore_chroma:
        print("\n🔍 Testando busca no Chroma:")
        query_teste = "introdução"
        resultados = vectorstore_chroma.similarity_search(query_teste, k=3)
        print(f"✅ Encontrados {len(resultados)} resultados para '{query_teste}'")
        print(f"\n📄 Primeiro resultado:")
        print("-" * 50)
        print(resultados[0].page_content[:200] + "...")
else:
    print("⚠️ Crie chunks e configure embeddings primeiro!")

### **4.2: Armazenamento no FAISS**

In [ ]:
# 🚀 ETAPA 4.2: ARMAZENAMENTO NO FAISS

def armazenar_faiss(chunks: List[Document], 
                   embeddings: HuggingFaceEmbeddings,
                   index_path: str = "./faiss_index"):
    """
    Armazena chunks no FAISS
    
    Args:
        chunks: Lista de chunks
        embeddings: Modelo de embeddings
        index_path: Caminho para salvar o índice
    
    Returns:
        Objeto FAISS configurado
    """
    print(f"🚀 Armazenando {len(chunks)} chunks no FAISS...")
    
    inicio = time.time()
    
    try:
        vectorstore = FAISS.from_documents(
            documents=chunks,
            embedding=embeddings
        )
        
        # Salvar índice
        vectorstore.save_local(index_path)
        
        tempo = time.time() - inicio
        
        print(f"✅ Armazenado no FAISS em {tempo:.2f}s")
        print(f"💾 Índice salvo em: {index_path}")
        
        return vectorstore
        
    except Exception as e:
        print(f"❌ Erro ao armazenar no FAISS: {e}")
        return None

def carregar_faiss(index_path: str, embeddings: HuggingFaceEmbeddings):
    """
    Carrega um índice FAISS existente
    """
    try:
        vectorstore = FAISS.load_local(
            folder_path=index_path,
            embeddings=embeddings
        )
        print(f"✅ FAISS carregado de {index_path}")
        return vectorstore
    except Exception as e:
        print(f"❌ Erro ao carregar FAISS: {e}")
        return None

# Armazenar no FAISS (se tiver chunks e embeddings)
if 'chunks_semanticos' in locals() and chunks_semanticos and embeddings:
    print("\n" + "="*60)
    vectorstore_faiss = armazenar_faiss(
        chunks_semanticos,
        embeddings,
        index_path="./faiss_index"
    )
    
    # Testar busca
    if vectorstore_faiss:
        print("\n🔍 Testando busca no FAISS:")
        query_teste = "conceitos principais"
        resultados = vectorstore_faiss.similarity_search(query_teste, k=3)
        print(f"✅ Encontrados {len(resultados)} resultados para '{query_teste}'")
        print(f"\n📄 Primeiro resultado:")
        print("-" * 50)
        print(resultados[0].page_content[:200] + "...")
else:
    print("⚠️ Crie chunks e configure embeddings primeiro!")

### **4.3: Armazenamento no LanceDB**

In [ ]:
# 📊 ETAPA 4.3: ARMAZENAMENTO NO LANCEDB

def armazenar_lancedb(chunks: List[Document], 
                     embeddings: HuggingFaceEmbeddings,
                     table_name: str = "pdfs_lancedb",
                     db_path: str = "./lancedb"):
    """
    Armazena chunks no LanceDB
    
    Args:
        chunks: Lista de chunks
        embeddings: Modelo de embeddings
        table_name: Nome da tabela
        db_path: Caminho do banco de dados
    
    Returns:
        Objeto LanceDB configurado
    """
    print(f"📊 Armazenando {len(chunks)} chunks no LanceDB...")
    
    inicio = time.time()
    
    try:
        # Conectar ao banco
        db = lancedb.connect(db_path)
        
        # Preparar dados
        dados = []
        
        print("📝 Gerando embeddings e preparando dados...")
        
        for i, chunk in enumerate(chunks):
            if i % 100 == 0:
                print(f"   Processando chunk {i+1}/{len(chunks)}")
            
            # Gerar embedding
            embedding = embeddings.embed_query(chunk.page_content)
            
            # Preparar dados
            dados.append({
                "vector": embedding,
                "text": chunk.page_content,
                "source": chunk.metadata.get('source_file', 'unknown'),
                "page": chunk.metadata.get('page_number', 0),
                "chunk_id": i
            })
        
        # Criar tabela
        print(f"🗄️ Criando tabela '{table_name}'...")
        
        if table_name in db.table_names():
            table = db.open_table(table_name)
            table.add(dados)
        else:
            table = db.create_table(table_name, dados)
        
        tempo = time.time() - inicio
        
        print(f"✅ Armazenado no LanceDB em {tempo:.2f}s")
        print(f"📊 Tabela: {table_name}")
        print(f"💾 Banco salvo em: {db_path}")
        
        return db, table
        
    except Exception as e:
        print(f"❌ Erro ao armazenar no LanceDB: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def buscar_lancedb(table, query: str, embeddings: HuggingFaceEmbeddings, k: int = 5):
    """
    Busca no LanceDB
    """
    query_embedding = embeddings.embed_query(query)
    
    resultados = table.search(query_embedding).limit(k).to_pandas()
    
    return resultados

# Armazenar no LanceDB (se tiver chunks e embeddings)
if 'chunks_semanticos' in locals() and chunks_semanticos and embeddings:
    print("\n" + "="*60)
    db_lancedb, table_lancedb = armazenar_lancedb(
        chunks_semanticos,
        embeddings,
        table_name="pdfs_longos_lancedb"
    )
    
    # Testar busca
    if table_lancedb:
        print("\n🔍 Testando busca no LanceDB:")
        query_teste = "resumo"
        resultados = buscar_lancedb(table_lancedb, query_teste, embeddings, k=3)
        print(f"✅ Encontrados {len(resultados)} resultados para '{query_teste}'")
        if len(resultados) > 0:
            print(f"\n📄 Primeiro resultado:")
            print("-" * 50)
            print(resultados.iloc[0]['text'][:200] + "...")
else:
    print("⚠️ Crie chunks e configure embeddings primeiro!")

## 🎯 **Pipeline Completo - Função Única**

In [ ]:
# 🎯 PIPELINE COMPLETO - FUNÇÃO ÚNICA

def pipeline_completo_ingestao_pdfs(
    pdf_folder: str = "pdfs",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    chunk_type: str = "semantico",  # "fixo", "semantico", "token"
    embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    vector_stores: List[str] = ["chroma", "faiss", "lancedb"]
) -> Dict[str, Any]:
    """
    Pipeline completo de ingestão de PDFs
    
    Args:
        pdf_folder: Pasta com PDFs
        chunk_size: Tamanho dos chunks
        chunk_overlap: Sobreposição entre chunks
        chunk_type: Tipo de chunk ("fixo", "semantico", "token")
        embedding_model: Modelo de embeddings
        vector_stores: Lista de bancos vetoriais a usar
    
    Returns:
        Dicionário com todos os vector stores criados
    """
    print("🚀 INICIANDO PIPELINE COMPLETO DE INGESTÃO")
    print("="*60)
    
    resultado = {}
    
    # Etapa 1: Carregar PDFs
    print("\n📄 ETAPA 1: Carregando PDFs...")
    documentos = carregar_todos_pdfs(pdf_folder)
    
    if not documentos:
        print("❌ Nenhum documento encontrado!")
        return resultado
    
    # Etapa 2: Criar chunks
    print("\n✂️ ETAPA 2: Criando chunks...")
    if chunk_type == "fixo":
        chunks = criar_chunks_fixos(documentos, chunk_size, chunk_overlap)
    elif chunk_type == "token":
        chunks = criar_chunks_tokens(documentos, chunk_size, chunk_overlap)
    else:  # semantico
        chunks = criar_chunks_semanticos(documentos, chunk_size, chunk_overlap)
    
    # Etapa 3: Configurar embeddings
    print("\n🧠 ETAPA 3: Configurando embeddings...")
    embeddings = configurar_embeddings(embedding_model)
    
    if not embeddings:
        print("❌ Erro ao configurar embeddings!")
        return resultado
    
    # Etapa 4: Armazenar em vector stores
    print("\n🗄️ ETAPA 4: Armazenando em vector stores...")
    
    if "chroma" in vector_stores:
        print("\n   📦 Armazenando no Chroma...")
        resultado["chroma"] = armazenar_chroma(chunks, embeddings)
    
    if "faiss" in vector_stores:
        print("\n   📦 Armazenando no FAISS...")
        resultado["faiss"] = armazenar_faiss(chunks, embeddings)
    
    if "lancedb" in vector_stores:
        print("\n   📦 Armazenando no LanceDB...")
        db, table = armazenar_lancedb(chunks, embeddings)
        resultado["lancedb"] = {"db": db, "table": table}
    
    print("\n" + "="*60)
    print("✅ PIPELINE COMPLETO FINALIZADO!")
    print(f"📊 Estatísticas:")
    print(f"   - PDFs processados: {len(set(d.metadata.get('source_file', '') for d in documentos))}")
    print(f"   - Páginas totais: {len(documentos)}")
    print(f"   - Chunks criados: {len(chunks)}")
    print(f"   - Vector stores: {len(resultado)}")
    
    return resultado

# Executar pipeline completo
print("💡 Para executar o pipeline completo, descomente a linha abaixo:")
print("# resultado_pipeline = pipeline_completo_ingestao_pdfs(")
print("#     pdf_folder='pdfs',")
print("#     chunk_size=1000,")
print("#     chunk_overlap=200,")
print("#     chunk_type='semantico',")
print("#     vector_stores=['chroma', 'faiss', 'lancedb']")
print("# )")

## 📊 **Resumo e Próximos Passos**

In [ ]:
# 📊 RESUMO DO QUE APRENDEMOS

print("🎓 RESUMO DO PIPELINE DE INGESTÃO")
print("="*60)
print("")
print("✅ Extração de texto de PDFs")
print("   - PyPDFLoader do LangChain")
print("   - Suporte para múltiplos PDFs")
print("   - Metadados preservados")
print("")
print("✅ Criação de chunks")
print("   - Chunks fixos (Character-based)")
print("   - Chunks semânticos (Recursive)")
print("   - Chunks baseados em tokens")
print("")
print("✅ Geração de embeddings")
print("   - HuggingFace Embeddings")
print("   - Modelos gratuitos")
print("")
print("✅ Armazenamento em vector stores")
print("   - Chroma (local, rápido)")
print("   - FAISS (escalável)")
print("   - LanceDB (moderno)")
print("")
print("🚀 PRÓXIMOS PASSOS:")
print("   1. Use o pipeline_completo_ingestao_pdfs()")
print("   2. Integre com seu sistema RAG")
print("   3. Configure busca semântica")
print("   4. Deploy em produção")
print("")
print("💡 Dicas:")
print("   - Ajuste chunk_size e chunk_overlap conforme seu caso")
print("   - Use chunks semânticos para melhor qualidade")
print("   - Teste diferentes modelos de embeddings")
print("   - Escolha o vector store baseado em suas necessidades")